In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# École Polytechnique de Montréal
Département Génie Informatique et Génie Logiciel
INF8460 – Traitement automatique de la langue naturelle

### Prof. Amal Zouaq
### Chargé de laboratoire: Félix Martel


# INF8460 - TP2

## Objectifs

•	Explorer les modèles d’espaces vectoriels comme représentations distribuées de la sémantique des mots et des documents

•	Comprendre différentes mesures de distance entre vecteurs de documents et de mots

•	Utiliser un modèle de langue n-gramme de caractères et l’algorithme Naive Bayes pour l’analyse de sentiments dans des revues de films (positives, négatives)


## 1. Prétraitement

Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données.

Commencez par lire ces données, en gardant séparées les données d'entraînement et de test.

In [11]:
from tqdm import trange, tqdm_notebook as tqdm

In [1]:
import os
data_path = './data/aclImdb/'

def data_reader(path):
    """ Cette fonction lit les données text et sauvegarde les labels. 
    
    :param path : str, Chemin vers le dossier contenant les données.
    :return X,y: list,list, X est une liste contenant les textes et y est une liste contenant les labels"""
    
    X = []
    y = []
    for label in ['pos','neg']:
        dir_path = path+label+'/'
        for filename in os.listdir(dir_path) :
            filepath = dir_path+filename
            with open(filepath,'r', encoding = 'utf-8') as file_descriptor:
                X.append(file_descriptor.read())
                y.append(label)
    return X,y

X_train, y_train = data_reader(data_path+'train/')
X_test, y_test = data_reader(data_path+'test/')

**a)** Créez la fonction `clean_doc()` qui effectue les pré-traitements suivants : segmentation en mots ; 
suppression des signes de ponctuations ; suppression des mots qui contiennent des caractères autres qu’alphabétiques ; 
suppression des mots qui sont connus comme des stop words ; suppression des mots qui ont une longueur de 1 caractère. Ensuite, appliquez-la à vos données.

Les stop words peuvent être obtenus avec `from nltk.corpus import stopwords`. Vous pourrez utiliser des [expressions régulières](https://docs.python.org/3.7/howto/regex.html).

In [22]:
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer

stop = stopwords.words("english")
reTokenizer = RegexpTokenizer('[A-Z|a-z]+')

def clean_doc(document):
    """Effectue les pré-traitements suivants  à un document : 
    segmentation en mots ; 
    suppression des signes de ponctuations ; 
    suppression des mots qui contiennent des caractères autres qu’alphabétiques ; 
    suppression des mots qui sont connus comme des stop words ; 
    suppression des mots qui ont une longueur de 1 caractère.
    
    :param sentence : str, le texte lu à partir d'un fichier de données.
    :return : list, liste de tokens traités.
    """
    
    #Segmentation en mots et suppression des signes de ponctuation
    tokens = reTokenizer.tokenize(document)
    #Suppression des stopwords
    tokens = [token for token in tokens if token.lower() not in stop]
    #Suppression des mots d'une longueur de 1 caractère
    tokens = [token for token in tokens if len(token) > 1]
    return tokens

def clean_corpus(corpus):
    """Effectue les pré-traitements suivants  à l'ensemble du corpus : 
    segmentation en mots ; 
    suppression des signes de ponctuations ; 
    suppression des mots qui contiennent des caractères autres qu’alphabétiques ; 
    suppression des mots qui sont connus comme des stop words ; 
    suppression des mots qui ont une longueur de 1 caractère.
    
    :param sentence : str, le texte lu à partir d'un fichier de données.
    :return : list, liste de tokens traités.
    """
    
    return [clean_doc(doc) for doc in corpus]
    

#Affichage d'un test
print(X_train[0])
print(clean_doc(X_train[0]))

#Application de clean_doc à l'ensemble du corpus
X_train = clean_corpus(X_train)
X_test = clean_corpus(X_test)

['Bromwell', 'High', 'cartoon', 'comedy', 'It', 'ran', 'time', 'programs', 'school', 'life', 'Teachers', 'My', 'years', 'teaching', 'profession', 'lead', 'believe', 'Bromwell', 'High', 'satire', 'much', 'closer', 'reality', 'Teachers', 'The', 'scramble', 'survive', 'financially', 'insightful', 'students', 'see', 'right', 'pathetic', 'teachers', 'pomp', 'pettiness', 'whole', 'situation', 'remind', 'schools', 'knew', 'students', 'When', 'saw', 'episode', 'student', 'repeatedly', 'tried', 'burn', 'school', 'immediately', 'recalled', 'High', 'classic', 'line', 'INSPECTOR', 'sack', 'one', 'teachers', 'STUDENT', 'Welcome', 'Bromwell', 'High', 'expect', 'many', 'adults', 'age', 'think', 'Bromwell', 'High', 'far', 'fetched', 'What', 'pity']


TypeError: expected string or bytes-like object

**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence de 5 au moins et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne).

In [3]:
from nltk.lm.vocabulary import Vocabulary
from nltk.lm.preprocessing import padded_everygram_pipeline

def build_voc(corpus):
    """
    Extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence de 5 au moins
    :param corpus: list(list(str)), un corpus tokenizé
    :return: None
    """
    ngrams, words = padded_everygram_pipeline(1, corpus)
    vocab = Vocabulary(words, unk_cutoff=5)
    print("Nombre de mots dans le vocabulaire:",len(vocab)-1) 
    
    with open('./output/vocab.txt',"w") as f:
        for word in list(vocab)[:-1]:
            f.write(word+'\n')
    f.close()
    return None

build_voc(X_train)

Nombre de mots dans le vocabulaire: 33162


**c)** Vous devez créer une fonction `get_top_unigrams(n)` qui retourne les $n$ unigrammes les plus fréquents et les affiche, puis l'appeler avec $n=10$.

In [4]:
def get_top_unigrams(corpus, n, unk_cutoff = 5):
    """
    Retourne les  𝑛  unigrammes les plus fréquents et les affiche
    :param corpus: list(list(str)), un corpus tokenizé
    :param n: int, nombre d'unigrammes à afficher
    :unk_cutoff: int, le seuil au-dessous duquel un mot est considéré comme inconnu et remplacé par <UNK>
    :return: list(tuple(str, int)), liste des top unigrams avec leur fréquence
    """
    ngrams, words = padded_everygram_pipeline(1, corpus)
    vocab = Vocabulary(words, unk_cutoff=unk_cutoff)
    most_commun = vocab.counts.most_common(n)
    print(most_commun)
    return [unigram[0] for unigram in most_commun]

most_commun_unigrams = get_top_unigrams(X_train,10)

[('br', 101870), ('The', 45087), ('movie', 43362), ('film', 39692), ('one', 24414), ('like', 19503), ('It', 18433), ('This', 14935), ('good', 14496), ('time', 12438)]


**d)**	Vous devez créer une fonction `get_top_unigrams_per_cls(n, cls)` qui retourne les $n$ unigrammes les plus fréquents de la classe `cls` (pos ou neg) et les affiche.

In [5]:
import numpy as np

def get_top_unigrams_per_cls(n, cls, X, y):
    """
    Retourne les  𝑛  unigrammes les plus fréquents de la classe cls (pos ou neg) et les affiche
    :param n: int, nombre d'unigrammes à afficher
    :param cls: str, classe à rechercher (pos ou neg)
    :param X: list(list(str)), un corpus tokenizé
    :param y: list(str), liste des classes
    :return: list(str), liste des top unigrams de la bonne classe
    """
    X_cls = list(np.array(X)[np.array(y) == cls])
    ngrams, words = padded_everygram_pipeline(1, X_cls)
    vocab = Vocabulary(words, unk_cutoff=1)
    most_commun = vocab.counts.most_common(n)
    print(most_commun)
    return [unigram[0] for unigram in most_commun]

**e)**	Affichez les 10 unigrammes les plus fréquents dans la classe positive :

In [6]:
top_unigrams_per_cls_pos = get_top_unigrams_per_cls(10, "pos", X_train, y_train)

[('br', 49234), ('The', 22581), ('film', 20665), ('movie', 18816), ('one', 12361), ('It', 9625), ('like', 8628), ('This', 7675), ('good', 7346), ('story', 6517)]


**f)**	Affichez les 10 unigrammes les plus fréquents dans la classe négative :

In [7]:
top_unigrams_per_cls_neg = get_top_unigrams_per_cls(10, "neg", X_train, y_train)

[('br', 52636), ('movie', 24546), ('The', 22506), ('film', 19027), ('one', 12053), ('like', 10875), ('It', 8808), ('This', 7260), ('good', 7150), ('bad', 7013)]


## 2. Matrices de co-occurence

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

### 2.1 Matrice document × mot et TF-IDF


Soit $X \in \mathbb{R}^{m \times n}$ une matrice de $m$ documents et $n$ mots, telle que $X_{i,j}$ contient la fréquence d'occurrence du terme $j$ dans le document $i$ :

$$\textbf{rowsum}(X, d) = \sum_{j=1}^{n}X_{dj}$$

$$\textbf{TF}(X, d, t) = \frac{X_{d,t}}{\textbf{rowsum}(X, d)}$$

$$\textbf{IDF}(X, t) = \log\left(\frac{m}{|\{d : X_{d,t} > 0\}|}\right)$$

$$\textbf{TF-IDF}(X, d, t) = \textbf{TF}(X, d, t) \cdot \textbf{IDF}(X, t)$$


En utilisant le même vocabulaire de 5 000 unigrammes, vous devez représenter les documents dans une matrice de co-occurrence document × mot $M(d, w)$  et les pondérer avec la mesure TF-IDF.

In [8]:
top_unigrams = get_top_unigrams(X_train,5000)

[('br', 101870), ('The', 45087), ('movie', 43362), ('film', 39692), ('one', 24414), ('like', 19503), ('It', 18433), ('This', 14935), ('good', 14496), ('time', 12438), ('would', 12307), ('story', 11567), ('really', 11339), ('see', 11100), ('even', 11019), ('much', 9465), ('well', 9103), ('get', 9089), ('bad', 8825), ('people', 8791), ('great', 8404), ('made', 8253), ('first', 8072), ('make', 7884), ('way', 7796), ('also', 7776), ('could', 7773), ('movies', 7446), ('But', 7338), ('think', 7144), ('characters', 7049), ('character', 6960), ('And', 6799), ('films', 6758), ('seen', 6633), ('watch', 6451), ('plot', 6438), ('two', 6421), ('acting', 6304), ('life', 6265), ('many', 6264), ('There', 6205), ('never', 6152), ('know', 6075), ('little', 6025), ('show', 5991), ('In', 5964), ('love', 5904), ('best', 5859), ('ever', 5739), ('better', 5631), ('end', 5490), ('If', 5368), ('He', 5355), ('say', 5351), ('scene', 5343), ('scenes', 5161), ('still', 5080), ('man', 4937), ('something', 4885), ('

In [9]:
def matrice_coocurrence_document_mot(X, top_unigrams):
    """
    Créé la matrice de co-occurrence document x mot
    :param X: list(list(str)), corpus à preprocess
    :param top_unigrams: list(str), top unigrams de train
    :return: list(list(int)), la matrice de co-occurrences
    """
    dico_top_unigrams = {}
    
    matrix_cooccurrence = np.zeros((len(X), len(top_unigrams)))
    for i in range(len(top_unigrams)):
        dico_top_unigrams[top_unigrams[i]] = i
    for k in range(len(X)):
        for i in range(len(X[k])):
            try:
                matrix_cooccurrence[k][dico_top_unigrams[X[k][i]]]+=1
            except:
                pass
                
    return matrix_cooccurrence

matrice_coocurrence_document_mot_train = matrice_coocurrence_document_mot(X_train, top_unigrams)
matrice_coocurrence_document_mot_test = matrice_coocurrence_document_mot(X_test, top_unigrams)

In [12]:
def calculate_TFIDF(X):
    """
    Pondération de la matrice de co-occurrence documents x mots
    :param X: list(list(int)), matrice de co-occurrence à pondérer
    :return: list(list(int)), matrice de co-occurrence pondéré selon la méthode TF-IDF
    """
    weighted_X = np.zeros_like(X)
    sum_row = np.sum(X, axis=1)
    nb_documents = len(X)
    d = np.count_nonzero(X, axis=0)
    idf = np.log(nb_documents/d)
    for i in trange(len(X)):
        for j in range(len(X[0])):
            if d[j]>0:
                weighted_X[i,j] = (X[i,j]/sum_row[i])*idf[j]
    return weighted_X

weighted_matrice_coocurrence_document_mot_train = calculate_TFIDF(matrice_coocurrence_document_mot_train)
weighted_matrice_coocurrence_document_mot_test = calculate_TFIDF(matrice_coocurrence_document_mot_test)

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:54<00:00, 218.31it/s]
C:\Users\Mehdi EL AYACHI\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:56<00:00, 214.37it/s]


### 2.2 Matrice mot × mot et PPMI (*positive pointwise mutual information*)

Vous devez calculer la métrique PPMI. Pour une matrice $m \times n$ $X$ :


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


**a)**	A partir des textes du corpus d’entrainement (neg *et* pos), vous devez construire une matrice de co-occurrence mot × mot $M(w,w)$ qui contient les 5000 unigrammes les plus fréquents. 

In [13]:
from tqdm import tqdm
def create_matrice_cooccurence_mot_mot(corpus, top_unigrams):
    dico_top_unigrams = {}
    cooccurrence_matrix = np.zeros((len(top_unigrams), len(top_unigrams)))
    for i in range(len(top_unigrams)):
        dico_top_unigrams[top_unigrams[i]] = i
        
    for sentence in tqdm(corpus):
        for i in range(len(sentence)):
            try:
                index_current_word = dico_top_unigrams[sentence[i]]
                for word in np.concatenate((sentence[max(0,i-5):i],sentence[i+1:i+6])):
                    try:
                        cooccurrence_matrix[index_current_word][dico_top_unigrams[word]]+=1
                    except:
                        pass
            except:
                pass
    return cooccurrence_matrix
matrice_coocurrence_mot_mot_train = create_matrice_cooccurence_mot_mot(X_train, top_unigrams)
matrice_coocurrence_mot_mot_test = create_matrice_cooccurence_mot_mot(X_test, top_unigrams)

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:48<00:00, 515.05it/s]


**b)**	Vous devez créer une fonction `calculate_PPMI` qui prend la matrice $M(w,w)$ et la transforme en une matrice $M’(w,w)$ avec les valeurs PPMI.

In [14]:
from tqdm import trange

def calculate_PPMI(X):
    """
    Pondération de la matrice de co-occurrence mots x mots
    :param X: list(list(int)), matrice de co-occurrence à pondérer
    :return: list(list(int)), matrice de co-occurrence pondéré selon la méthode PPMI
    """
    weighted_X = np.zeros_like(X)
    sum_col = np.sum(X, axis=0)
    sum_row = np.sum(X, axis=1)
    sum_X = np.sum(X)
    weights = np.transpose(np.dot(np.transpose([sum_col]),[sum_row]))/sum_X
    for i in trange(len(X)):
        for j in range(len(X[0])):
            if weights[i,j]!=0 and X[i,j]>weights[i,j]:
                weighted_X[i,j] = np.log(X[i,j]/weights[i,j])
    return weighted_X

weighted_matrice_coocurrence_mot_mot_train = calculate_PPMI(matrice_coocurrence_mot_mot_train)
weighted_matrice_coocurrence_mot_mot_test = calculate_PPMI(matrice_coocurrence_mot_mot_test)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:31<00:00, 159.10it/s]


## 3. Mesures de similarité

En utilisant le module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html),  définissez des fonctions pour calculer les métriques suivantes :

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**a)** Implémentez la fonction `get_euclidean_distance(v1 ,v2)` qui retourne la distance euclidienne entre les vecteurs v1 et v2.

In [15]:
from scipy.spatial.distance import euclidean

def get_euclidean_distance(v1, v2):
    """
    Cettefonction calcule la distance euclidienne entre deux 
    vecteurs v1 et v2.
    :param v1 : 1-D array, vecteur 1
    :param v2 : 1-D array, vecteur 2
    :return : float, distance entre les vecteurs v1 et v2
    """
    
    return euclidean(v1,v2)

#Affichage d'un test :
v1, v2 = ([0,1,2,3],[1,2,3,4])
print('v1 : ', v1)
print('v2 : ', v2)
print(get_euclidean_distance(v1,v2))

v1 :  [0, 1, 2, 3]
v2 :  [1, 2, 3, 4]
2.0


**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

**b)** Implémentez la fonction `get_cosinus_distance(v1, v2)` qui retourne la distance cosinus entre les vecteurs v1 et v2.

In [16]:
from scipy.spatial.distance import cosine

def get_cosinus_distance(v1, v2):
    """
    Cettefonction calcule la distance cosinus entre deux 
    vecteurs v1 et v2.
    :param v1 : 1-D array, vecteur 1
    :param v2 : 1-D array, vecteur 2
    :return : float, distance entre les vecteurs v1 et v2
    """
    
    return cosine(v1,v2)

#Affichage d'un test :
v1, v2 = ([0,1,2,3],[0,2,4,6])
print('v1 : ', v1)
print('v2 : ', v2)
print(get_cosinus_distance(v1,v2))

v1 :  [0, 1, 2, 3]
v2 :  [0, 2, 4, 6]
0.0


**c)** Implémentez la fonction `get_most_similar_PPMI(word, metric, n)` qui prend un mot en entrée et une mesure de distance et qui retourne les n mots les plus similaires selon la mesure. Les mesures à tester sont : la distance euclidienne et la distance cosinus implantées ci-dessus. Le vecteur du mot word doit être extrait de la matrice $M’(w,w)$.

In [17]:
import heapq

def get_most_similar_PPMI(word, metric, n):
    priority_queue = []
    index_word = top_unigrams.index(word)
    for i in range(len(weighted_matrice_coocurrence_mot_mot_train)):
        if top_unigrams[i] != word:
            heapq.heappush(priority_queue, (metric(weighted_matrice_coocurrence_mot_mot_train[index_word],weighted_matrice_coocurrence_mot_mot_train[i]), top_unigrams[i]))
    result = []
    for i in range(n):
        result.append(heapq.heappop(priority_queue)[1])
    return result

**d)** Trouvez les 5 mots les plus similaires au mot « bad » et affichez-les, pour chacune des deux distances. Commentez.

In [18]:
n = 10
print("Les 5 mots les plus similaires à 'bad' avec la distance euclidean :")
print(get_most_similar_PPMI('bad', get_euclidean_distance, n))
print()
print("Les 5 mots les plus similaires à 'bad' avec la distance cosinus :")
print(get_most_similar_PPMI('bad', get_cosinus_distance, n))

Les 5 mots les plus similaires à 'bad' avec la distance euclidean :
['movie', 'good', 'br', 'really', 'It', 'film', 'This', 'one', 'even', 'The']

Les 5 mots les plus similaires à 'bad' avec la distance cosinus :
['awful', 'terrible', 'acting', 'horrible', 'good', 'poor', 'stupid', 'movie', 'script', 'cheesy']


-> Commentez ici <-

**e)** Implémentez la fonction `get_most_similar_TFIDF(word, metric, n)` qui prend un mot en entrée et une mesure de distance et qui retourne les n mots les plus similaires selon la mesure. Les mesures à tester sont : la distance euclidienne et la distance cosinus implantées ci-dessus. Le vecteur du mot word doit être extrait de la matrice $M(d,w)$.

In [19]:
def get_most_similar_TFIDF(word, metric, n):
    priority_queue = []
    index_word = top_unigrams.index(word)
    for i in range(len(top_unigrams)):
        if top_unigrams[i] != word:
            heapq.heappush(priority_queue, (metric(weighted_matrice_coocurrence_document_mot_train[:,index_word],weighted_matrice_coocurrence_document_mot_train[:,i]), top_unigrams[i]))
    result = []
    for i in range(n):
        result.append(heapq.heappop(priority_queue)[1])
    return result

**f)** Trouvez les 5 mots les plus similaires au mot « bad » et affichez-les, pour chacune des deux distances. Commentez

In [20]:
n = 10
print("Les 5 mots les plus similaires à 'bad' avec la distance euclidean :")
print(get_most_similar_TFIDF('bad', get_euclidean_distance, n))
print()
print("Les 5 mots les plus similaires à 'bad' avec la distance cosinus :")
print(get_most_similar_TFIDF('bad', get_cosinus_distance, n))

Les 5 mots les plus similaires à 'bad' avec la distance euclidean :
['The', 'mentions', 'crucial', 'arrival', 'chest', 'aforementioned', 'introduces', 'Secondly', 'Directed', 'namely']

Les 5 mots les plus similaires à 'bad' avec la distance cosinus :
['movie', 'acting', 'The', 'good', 'br', 'This', 'like', 'one', 'even', 'worst']


*-> Commentez ici <-*

## 4. Classification de documents avec un modèle de langue

En vous inspirant de [cet article](https://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139), entraînez deux modèles de langue $n$-gramme de caractère avec lissage de Laplace, l'un sur le corpus `pos`, l'autre sur le corpus `neg`. Puis, pour chaque document $D$, calculez sa probabilité selon vos deux modèles : $P(D \mid \textrm{pos})$ et $P(D \mid \textrm{neg})$.

Vous pourrez alors prédire sa classe $\hat{c}_D \in (\textrm{pos}, \textrm{neg})$ en prenant :

$$\hat{c}_D = \begin{cases}
\textrm{pos} & \textrm{si } P(D \mid \textrm{pos}) > P(D \mid \textrm{neg}) \\
\textrm{neg} & \textrm{sinon}
\end{cases}$$

In [23]:
#Training models
import nltk
import string
from tqdm import tqdm_notebook as tqdm
from nltk.util import ngrams
from nltk.lm.preprocessing import pad_both_ends
import numpy as np



#Ordre de notre model
order = 4

#Creation du vocabulaire 
vocab = nltk.lm.vocabulary.Vocabulary([chr(i) for i in range(ord('a'), ord('a')+27)]+[' '])

#Separation des données 'pos' et 'neg' :
data, n_grams, laplace = {},{},{}
for label in ['pos', 'neg']:
    print('Separation des données du set : ',label)
    data[label] = np.array(X_train)[np.array(y_train)==label]
    #print('Transformation en minuscule et suppression de la ponctuation pour le set : '+label)
    #data[label] = [doc.translate(str.maketrans('', '', string.punctuation)).lower() for doc in tqdm(data[label])]
    print('Transformation en minuscule et suppression de la ponctuation pour le set : '+label)
    data[label] = [" ".join(doc).translate(str.maketrans('', '', string.punctuation)).lower() for doc in tqdm(data[label])]
    print('Collecte des ngrams pour le "'+label+'" set :')
    n_grams[label] = [ngrams(pad_both_ends(document, order), order) for document in tqdm(data[label])]
    laplace[label] = nltk.lm.models.Laplace(order, vocab)
    print('Fitting du model laplace pour le set : '+label)
    laplace[label].fit(n_grams[label])



Separation des données
Transformation en minuscule et suppression de la ponctuation pour le set : pos


HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))


Collecte des ngrams pour le "pos" set :


HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))


Fitting du model laplace pour le set : pos
Separation des données
Transformation en minuscule et suppression de la ponctuation pour le set : neg


HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))


Collecte des ngrams pour le "neg" set :


HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))


Fitting du model laplace pour le set : neg


In [24]:
import math
def laplace_classifier(doc, order = order):
    #Cleaning doc from punctuation and tokenizing it
    tokens = [ch for ch in " ".join(doc).translate(str.maketrans('', '', string.punctuation)).lower()]
    
    #Computing perplexity 
    perplexity_pos = 0
    perplexity_neg = 0
    n_grams = list(ngrams(tokens, order, pad_right=True, pad_left=True))
    probs = {"pos":0,"neg":0}
    for ngram in n_grams:
        for label in probs.keys() :
            probs[label] += math.log10(laplace[label].unmasked_score(ngram[-1], ngram[0:-1]))
    
    classe = ["neg","pos"][probs["pos"] > probs["neg"]]
    return classe

In [25]:
print("Testing Laplace Classifier :")
for i in range(5):
    print("Test ",i+1," :\n    Predicted classe : " ,laplace_classifier(X_test[i]),"\t Target : ",y_test[i])

Testing Laplace Classifier :
Test  1  :
    Predicted classe :  pos 	 Target :  pos
Test  2  :
    Predicted classe :  pos 	 Target :  pos
Test  3  :
    Predicted classe :  pos 	 Target :  pos
Test  4  :
    Predicted classe :  pos 	 Target :  pos
Test  5  :
    Predicted classe :  pos 	 Target :  pos


## 5. Classification de documents avec sac de mots et Naive Bayes

Ici, vous utiliserez l'algorithme Multinomial Naive Bayes (disponible dans [`sklearn.naive_bayes.MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)) pour classifier les documents. Vous utiliserez un modèle sac de mots (en anglais *bag of words*, ou BoW) avec TF-IDF pour représenter vos documents.

*Note :* vous avez déjà construit la matrice TF-IDF à la section 2.1.

In [26]:
from sklearn.naive_bayes import MultinomialNB
mnb_model = MultinomialNB()
mnb_model.fit(weighted_matrice_coocurrence_document_mot_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
print("Testing Multinomial Naive Bayes Classifier :")
for i in range(5):
    print("Test ",i+1," :\n    Predicted classe : " ,
          mnb_model.predict([weighted_matrice_coocurrence_document_mot_test[i]])[0],
          "\t Target : ",y_test[i])

Testing Multinomial Naive Bayes Classifier :
Test  1  :
    Predicted classe :  pos 	 Target :  pos
Test  2  :
    Predicted classe :  pos 	 Target :  pos
Test  3  :
    Predicted classe :  pos 	 Target :  pos
Test  4  :
    Predicted classe :  pos 	 Target :  pos
Test  5  :
    Predicted classe :  pos 	 Target :  pos


## 6. Améliorations

Ici, vous devez proposer une méthode d'amélioration pour le modèle précédent, la justifier et l'implémenter.

*-> Écrivez vos explications ici <-*

In [62]:
from sklearn.model_selection import GridSearchCV

parameters = {'alpha': [0]+[10**i for i in range(10)]}
optimised_mnb = MultinomialNB()
gs = GridSearchCV(estimator=optimised_mnb,
                     param_grid = parameters,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)

# On change la liste des targets pour reduire l'utilisation de mémoire du GridSearch
# Sinon on obtient l'exception MemoryError
labels_train = [1 if label=='pos' else 0 for label in y_train]
gs.fit(weighted_matrice_coocurrence_document_mot_train, labels_train)
print('Meilleur score de precision :', gs.best_score_)
print('Meilleur parametre : ', gs.best_params_)


Meilleur score de precision : 0.8408
Meilleur parametre :  {'alpha': 10}


In [63]:
print("Testing Optimised Multinomial Naive Bayes Classifier :")
for i in range(5):
    prediction = 'pos' if gs.predict([weighted_matrice_coocurrence_document_mot_test[i]])[0]==1 else 'neg'
    print()
    print("Test ",i+1," :\n    Predicted classe : " ,
          prediction,
          "\t Target : ",y_test[i])

Testing Optimised Multinomial Naive Bayes Classifier :
Test  1  :
    Predicted classe :  pos 	 Target :  pos
Test  2  :
    Predicted classe :  pos 	 Target :  pos
Test  3  :
    Predicted classe :  pos 	 Target :  pos
Test  4  :
    Predicted classe :  pos 	 Target :  pos
Test  5  :
    Predicted classe :  pos 	 Target :  pos


## 7. Évaluation

Évaluation des modèles des sections 4, 5, 6 sur les données de test. On attend les métriques suivantes : *accuracy*, et pour chaque classe précision, rappel, score F1. Vous pourrez utiliser le module [`sklearn.metrics`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [67]:
from sklearn.metrics import classification_report, confusion_matrix
import sklearn
#predictions = {}
#predictions["Vanilla Laplace"] = [laplace_classifier(doc)for doc in tqdm(X_test)]
#predictions["MNB"] = mnb_model.predict(weighted_matrice_coocurrence_document_mot_test)
#predictions["Laplace Amélioré"] = ['pos' if label==1 else 'neg'
#                                   for label in gs.predict(weighted_matrice_coocurrence_document_mot_test)]

for model_name in predictions.keys():
    print("Test du classifieur ",model_name," : ")
    print(sklearn.metrics.classification_report(y_test, predictions[model_name])) 
    print(sklearn.metrics.confusion_matrix(y_test, predictions[model_name]), columns=["neg", "pos"], index=["neg", "pos"])

Test du classifieur  Vanilla Laplace  : 
             precision    recall  f1-score   support

        neg       0.80      0.86      0.83     12500
        pos       0.85      0.78      0.81     12500

avg / total       0.82      0.82      0.82     25000



TypeError: 'columns' is an invalid keyword argument for this function

Commentez vos résultats.

*-> Commentez ici vos résultats <-*